In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
# Задание 1
# Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
# Вам необходимо собирать только те статьи, в которых встречается хотя бы
# одно требуемое ключевое слово. Эти слова определяем в начале кода в
# переменной, например:
# KEYWORDS = ['python', 'парсинг']
# Поиск вести по всей доступной preview-информации (это информация,
# доступная непосредственно с текущей страницы).
# В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

KEYWORDS = ['python', 'парсинг']

import re
from datetime import date, timedelta

from pymystem3 import Mystem
from string import punctuation

m = Mystem()
def lemmas(text):
    tokens = m.lemmatize(text.lower())
    tokens = [token for token in tokens if token != " " \
              and token.strip() not in punctuation]
    return set(tokens)

def article_contains_keewords(article):
    kw = set(KEYWORDS)
    title = article.find('a', class_='post__title_link')
    title_text = title.get_text()
    url = (title_text, title.attrs['href'])
    # исследуем заголовок
    if lemmas(title_text) & kw:
        return url
    # исследуем названия хабов
    hubs = article.find('ul', class_='post__hubs')
    if lemmas(hubs.get_text()) & kw:
        return url
    # исследуем превью поста
    post = article.find('div', class_='post__text')
    if lemmas(post.get_text()) & kw:
        return url
    return None

def article_date(timestr):
    # (сегодня|вчера) в HH:MM или 26 ноября 2020 в 15:57
    dt = date.today()
    try:
        match = re.match(r'(.+) в \d\d:\d\d$', timestr)
        if match[1] == 'сегодня':
            pass
        elif match[1] == 'вчера':
            dt -= timedelta(days=1)
        else:
            MON = {
                'янв': 1, 'фев': 2, 'мар': 3, 
                'апр': 4, 'мая': 5, 'июн': 6, 
                'июл': 7, 'авг': 8, 'сен': 9, 
                'окт': 10, 'ноя': 11, 'дек': 12, 
            }
            day, mon, year = match[1].split(' ')
            dt = date(int(year), MON[mon[:3]], int(day))
    except:
        pass
    return dt


page = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(page.text, 'lxml')

data = pd.DataFrame(columns=['date', 'title', 'url'])

articles = soup.find_all('article', class_='post')
for article in articles:
    url = article_contains_keewords(article)
    if not url:
        continue
    timestr = article.find('span', class_='post__time').get_text()
    data.loc[len(data)] = [article_date(timestr), url[0], url[1]]

data

In [ ]:
# Задание 2
# Написать скрипт, который будет проверять список e-mail адресов на утечку
# при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной
# в начале кода:
# EMAIL = [xxx@x.ru, yyy@y.com]
# В итоге должен формироваться датафрейм со столбцами:
# <почта> - <дата утечки> - <источник утечки> - <описание утечки>
# Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите
# post-запросы.

# POST запрос
# Request (JSON):
# {
#    "emailAddresses": ["bs@artcon.ru", ]
# }
import json
import requests


EMAIL = ['7966678@mail.ru', 'karaulov@podarino.com', 'anry@artcon.ru']

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'Content-Type': 'application/json;charset=utf-8',
    'Accept': 'application/json',
    'Referer': 'https://www.avast.com/hackcheck',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
}

r = requests.post(url, headers=headers, json={'emailAddresses': EMAIL})

j = r.json()
data = pd.DataFrame(columns=['email', 'publishDate', 'site', 'description'])

breaches = j['breaches']
for email, val in j['summary'].items():
    for breachId in val['breaches']:
        breach = breaches[str(breachId)]
        data.loc[len(data)] = [
            email,
            datetime.strptime(
                breach['publishDate'],'%Y-%m-%dT%H:%M:%SZ').date(),
            breach['site'],
            breach['description']
        ]

data